In [0]:
%sql
DELETE FROM case_study_de.customer_bronze;
WITH customer_cte_v1 
(customer_nk, first_name, last_name, phone, country, email_id) 
AS (SELECT * FROM csv.`/Volumes/source_files/default/files/Customer_1.csv`),
customer_cte_v2 
(customer_nk, first_name, last_name, phone, country, email_id) 
AS (SELECT * FROM csv.`/Volumes/source_files/default/files/Customer_2.csv`),
customer_cte_v3 
(customer_nk, first_name, last_name, phone, country, email_id) 
AS (SELECT * FROM csv.`/Volumes/source_files/default/files/Customer_3.csv`),
customer_cte_v1_versioned AS 
(SELECT *, 1 as customer_version FROM customer_cte_v1),
customer_cte_v2_versioned AS 
(SELECT *, 2 as customer_version FROM customer_cte_v2),
customer_cte_v3_versioned AS 
(SELECT *, 3 as customer_version FROM customer_cte_v3),
customer_union AS (
  SELECT * FROM customer_cte_v1_versioned 
  UNION ALL 
  SELECT * FROM  customer_cte_v2_versioned
  UNION ALL 
  SELECT * FROM  customer_cte_v3_versioned
  ),
customer_transformed AS (SELECT 
  customer_version, customer_nk , first_name, last_name, phone, country, email_id,
  TRY_CAST(REPLACE(customer_nk, 'CUST', '') AS INT) AS customer_nk_num 
FROM customer_union),
customer_cte_filtered AS 
(SELECT * FROM customer_transformed WHERE customer_nk_num IS NOT NULL)
INSERT INTO case_study_de.customer_bronze (
  customer_nk,
  first_name,
  last_name,
  phone,
  country,
  email_id ,
  custmer_nk_num,
  customer_version
)
SELECT customer_nk, first_name, last_name, phone, country, email_id, customer_nk_num, customer_version   
FROM customer_cte_filtered;

